In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

In [2]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory( # summarize the old conversation after token limit (mix of both)
    llm=llm,
    max_token_limit=150,
    memory_key="history", # used to load memory variables in llmchain
    return_messages=True
)

examples = [
    {
        "input": "Top Gun",
        "output": "🛩️👨‍✈️🔥",
    },
    {
        "input": "The Godfather",
        "output": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["input"],
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful AI talking to a human. 
    If human gives you a movie, reply with three emojis that represent the movie.
    Refer to the examples below for guidance.
    """),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human", "{input}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm # output of load_memory is assigned to history when invoked

def invoke_chain(input):
    answer = chain.invoke({"input": input})
    memory.save_context(
        {"input": input},
        {"output": answer.content},
    )
    print(answer.content)

invoke_chain("Inception")
invoke_chain("John Wick")
invoke_chain("The Notebook")
invoke_chain("what is the movie I asked you first?")

🕰️💭🔁
🔫🐶🚗
💑📒💔
The first movie you asked about was "Inception."
